# Using KDB+
* * *
## Load Database
### From Original CSVs

This action only has to be performed once. If you already have the splayed tables saved somewhere, go to section "From Splayed Tables" to load them.

In [2]:
/ Load price tick file
tick:("SIDTFIFIFIS"; enlist"|")0:`:/Users/alfredo.leon/Desktop/findata/data/scale_1000/tick_price_file_no_spaces.csv
/ Create enumeration for table (this is required to create a splayed table and then a partitioned table)
tickenum: .Q.en[`:../data/test/raw/tickprice/] tick
/ Sample query
select[10] from tickenum where TimeStamp within 09:00:00.00 09:00:10.00
/ Save table
/rsave `tickenum

Id           SeqNo TradeDate  TimeStamp    TradePrice TradeSize AskPrice AskS..
-----------------------------------------------------------------------------..
Security_359 1     2022.11.21 09:00:00.000 43         6300                   ..
Security_667 1     2022.11.21 09:00:00.000 57         9100                   ..
Security_588 1     2022.11.21 09:00:00.000 27         6800                   ..
Security_371 1     2022.11.21 09:00:00.000                      23.0938  9600..
Security_661 1     2022.11.21 09:00:01.000                                   ..
Security_591 1     2022.11.21 09:00:01.000                                   ..
Security_794 1     2022.11.21 09:00:01.000 26         700                    ..
Security_602 1     2022.11.21 09:00:01.000 58         8100                   ..
Security_539 1     2022.11.21 09:00:02.000                                   ..
Security_27  1     2022.11.21 09:00:02.000 31         5600                   ..


In [2]:
select last SeqNo from tickenum

SeqNo
-----
583  


In [6]:
/ Load base tick file
base:("SSSSS"; enlist"|")0:`:/Users/alfredo.leon/Desktop/findata/data/scale_1000/tick_base_file_no_spaces.csv
/ Create enumeration for table (this is required to create a splayed table and then a partitioned table)
baseenum: .Q.en[`:../data/test/raw/basetick/] base
/ Sample query
select[5] from baseenum
/ Save table
/rsave `baseenum

Id         Ex Descr                          SIC             Cu 
----------------------------------------------------------------
Security_0 LN 'Financial security number: 0' COMPUTERS       FFR
Security_1 NY 'Financial security number: 1' PHARMACEUTICALS FFR
Security_2 O  'Financial security number: 2' SOFTWARE        USD
Security_3 O  'Financial security number: 3' SOFTWARE        USD
Security_4 AM 'Financial security number: 4' COMPUTERS       GBP


### From Splayed Tables

In [ ]:
/ Load the tables from saved splayed table using get to save into variable.
/baseenum2:get `baseenum / This should be the path to the foldeer with data.
/priceenum2:get `tickenum

In [ ]:
/ Pending: Figure out how to partition tables and save/load those

## Benchmark Queries

1. Get all ticks for a specified set of 100 securities for a specified three hour time period on a specified trade date.

2. Determine the volume weighted price of a security considering only the ticks in a specified three hour interval

3. Determine the top 10 percentage losers for the specified date on the specified exchanges sorted by percentage loss. The loss is calculated as a percentage of the last trade price of the previous day.

4. Determine the top 10 most active stocks for a specified date sorted by cumulative trade volume by considering all trades

5. Find the most active stocks in the "COMPUTER" industry (use SIC code)

6. Find the 10 stocks with the highest percentage spreads. Spread is the difference between the last ask-price and the last bid-price. Percentage spread is calculated as a percentage of the mid-point price (average of ask and bid price).

1. Get all ticks for a specified set of 100 securities for a specified three hour time period on a specified trade date.

In [2]:
distinct select TradeDate from tickenum

TradeDate 
----------
2022.11.21
2022.11.22
2022.11.23
2022.11.24
2022.11.25


In [2]:
secs : select[101] Id from tickenum
securities: (exec Id from secs)  / This is a list
select from tickenum where Id in securities, TradeDate=2022.11.21, TimeStamp within 09:00:00.00 12:00:00.00

Id           SeqNo TradeDate  TimeStamp    TradePrice TradeSize AskPrice AskS..
-----------------------------------------------------------------------------..
Security_359 1     2022.11.21 09:00:00.000 43         6300                   ..
Security_667 1     2022.11.21 09:00:00.000 57         9100                   ..
Security_588 1     2022.11.21 09:00:00.000 27         6800                   ..
Security_371 1     2022.11.21 09:00:00.000                      23.0938  9600..
Security_661 1     2022.11.21 09:00:01.000                                   ..
Security_591 1     2022.11.21 09:00:01.000                                   ..
Security_794 1     2022.11.21 09:00:01.000 26         700                    ..
Security_602 1     2022.11.21 09:00:01.000 58         8100                   ..
Security_539 1     2022.11.21 09:00:02.000                                   ..
Security_27  1     2022.11.21 09:00:02.000 31         5600                   ..
Security_634 1     2022.11.21 09:00:02.0

2. Determine the volume weighted price of a security considering only the ticks in a specified three hour interval.

In [2]:
select sum(TradePrice * TradeSize) % sum(TradeSize) from tickenum where Id=`Security_1, TradeDate=2022.11.21, TimeStamp within 09:00:00.00 12:00:00.00

TradePrice
----------
3.99304   


3. Determine the top 10 percentage losers for the specified date on the specified exchanges sorted by percentage loss. The loss is calculated as a percentage of the last trade price of the previous day.

In [53]:
select[10] from
    `LossPercentage xasc `Id`LossPercentage xcol
    select Id, (ClosingPriceToday-ClosingPricePreviousDay)*100%ClosingPricePreviousDay from(
        `Id`ClosingPriceToday`ClosingPricePreviousDay xcol
        (
            (`Id`ClosingPriceToday xcol select first TradePrice by Id from (`TimeStamp xdesc 
                select Id, TradePrice, TimeStamp 
                from tickenum
                where TradeDate=2022.11.22, TradePrice>0.0)) ij
            (`Id`ClosingPricePreviousDay xcol select first TradePrice by Id from (`TimeStamp xdesc 
                select Id, TradePrice, TimeStamp 
                from tickenum 
                where TradeDate=2022.11.23, TradePrice>0.0))
        )
    )

Id           LossPercentage
---------------------------
Security_944 -70           
Security_141 -62.5         
Security_965 -59.09091     
Security_751 -33.33333     
Security_515 -26.66667     
Security_475 -25.80645     
Security_10  -23.52941     
Security_921 -21.05263     
Security_194 -19.79167     
Security_405 -15.58442     


4. Determine the top 10 most active stocks for a specified date sorted by cumulative trade volume by considering all trades

In [5]:
select [10;>TradeCumulative] TradeCumulative:sum TradeSize by Id from tickenum where TradeDate=2022.11.22

Id          | TradeCumulative
------------| ---------------
Security_350| 379300         
Security_570| 376800         
Security_179| 348900         
Security_163| 348400         
Security_436| 348300         
Security_527| 347800         
Security_407| 345100         
Security_749| 345100         
Security_714| 344100         
Security_733| 341200         


5. Find the most active stocks in the "COMPUTER" industry (use SIC code)

In [52]:
select[10] from `TicksCount xdesc `Id`TicksCount xcol select sum TradeSize by Id from (
    ej[`Id;select from baseenum where SIC like "COMPUTERS"; tickenum])

Id          | TicksCount
------------| ----------
Security_471| 1506000   
Security_276| 1496600   
Security_683| 1486300   
Security_531| 1474800   
Security_934| 1463900   
Security_729| 1461100   
Security_773| 1455600   
Security_302| 1443600   
Security_846| 1441900   
Security_289| 1441700   


6. Find the 10 stocks with the highest percentage spreads. Spread is the difference between the last ask-price and the last bid-price. Percentage spread is calculated as a percentage of the mid-point price (average of ask and bid price).

In [57]:
select [10;>2*(AskPrice-BidPrice)%(AskPrice+BidPrice)] Id, Percentage_spread: 2*(AskPrice-BidPrice)%(AskPrice+BidPrice) from select  last AskPrice where not null AskPrice, last BidPrice where not null BidPrice by Id from tickenum where TradeDate=2022.11.23

Id           Percentage_spread
------------------------------
Security_944 0.3333333        
Security_808 0.2702703        
Security_141 0.2              
Security_355 0.1538462        
Security_751 0.1538462        
Security_965 0.1276596        
Security_489 0.08695652       
Security_519 0.08695652       
Security_921 0.07594937       
Security_238 0.07407407       


In [62]:
select[10] from `TradeSize xdesc select[10] from select sum(TradeSize) by Id from tickenum where TradeDate=2022.11.23

Id                            | TradeSize
------------------------------| ---------
INDUSTRIAL                    | 394300   
COMPUTERS                     | 350600   
'Financial security number: 0'| 289600   
'Financial security number: 2'| 288000   
'Financial security number: 4'| 264100   
'Financial security number: 3'| 249100   
FFR                           | 245900   
ENTERTAINMENT                 | 240900   
'Financial security number: 1'| 238200   
CHEMICALS                     | 188400   


In [74]:
secs : select[101] Id from baseenum
securities: (exec Id from secs)
select from tickenum where Id in securities, TradeDate=2022.11.21, TimeStamp within 09:00:00.00 12:00:00.00

Id           SeqNo TradeDate  TimeStamp    TradePrice TradeSize AskPrice AskS..
-----------------------------------------------------------------------------..
Security_485 1     2022.11.21 09:00:03.000                      12.0625  8900..
Security_493 1     2022.11.21 09:00:03.000                      44.0938  4300..
Security_171 1     2022.11.21 09:00:03.000                                   ..
Security_387 1     2022.11.21 09:00:04.000                                   ..
Security_337 1     2022.11.21 09:00:04.000 61         7300                   ..
Security_95  1     2022.11.21 09:00:04.000                      1        8100..
Security_815 1     2022.11.21 09:00:04.000 86         6600                   ..
Security_300 1     2022.11.21 09:00:05.000 23         9200                   ..
Security_300 2     2022.11.21 09:00:05.000                      23.0312  1400..
Security_384 1     2022.11.21 09:00:05.000 15         9100                   ..
Security_157 1     2022.11.21 09:00:06.0

In [5]:
0!select[10] from `Percentage_spread xdesc select Id, Percentage_spread: 2*(AskPrice-BidPrice)%(AskPrice+BidPrice) from select  last AskPrice where not null AskPrice, last BidPrice where not null BidPrice by Id from tickenum where TradeDate=2022.11.23


Id           Percentage_spread
------------------------------
Security_382 0.3333333        
Security_720 0.2702703        
Security_96  0.2              
Security_322 0.1538462        
Security_874 0.1538462        
Security_999 0.1276596        
Security_241 0.08695652       
Security_756 0.08695652       
Security_19  0.07594937       
Security_378 0.07407407       


In [7]:
0!select[10] from `Percentage_spread xdesc `Id`Percentage_spread xcol select Id, (2*(AskPrice-BidPrice)%(AskPrice+BidPrice)) from (select  last AskPrice where not null AskPrice, last BidPrice where not null BidPrice by Id from tickenum where TradeDate=2022.11.22)


Id           Percentage_spread
------------------------------
Security_944 0.2857143        
Security_965 0.2              
Security_899 0.1290323        
Security_751 0.1052632        
Security_355 0.08695652       
Security_519 0.08695652       
Security_316 0.06134969       
Security_10  0.04878049       
Security_534 0.04580153       
Security_913 0.04444444       


In [9]:
0!`Id`Percentage_spread xcol select Id, (2*(AskPrice-BidPrice)%(AskPrice+BidPrice)) from select last AskPrice where not null AskPrice, last BidPrice where not null BidPrice by Id from tickenum where TradeDate=2022.11.22


Id                             Percentage_spread
------------------------------------------------
'Financial security number: 0' 0.002186221      
'Financial security number: 1' 0.001640666      
'Financial security number: 2' 0.002244669      
'Financial security number: 3' 0.001341036      
'Financial security number: 4' 0.004336086      
CHEMICALS                      0.002012903      
COMPUTERS                      0.002354717      
ENTERTAINMENT                  0                
FFR                            0                
INDUSTRIAL                     0                
JPY                            -0.007168459     
PHARMACEUTICALS                0.009569378      
Security_0                     -0.007782101     
Security_1                     0.0006533812     
Security_10                    0.04878049       
Security_100                   -0.0008877053    
Security_101                   0.001352265      
Security_102                   0                
Security_103        